In [4]:
from collections import defaultdict

default_dict = defaultdict(lambda: None)
default_dict['test'] is None

True

In [6]:
from transformers import PretrainedConfig

PretrainedConfig.get_config_dict(model_name)

loading configuration file config.json from cache at /home/tminh/.cache/huggingface/hub/models--bkai-foundation-models--vietnamese-bi-encoder/snapshots/84f9d9ada0d1a3c37557398b9ae9fcedcdf40be0/config.json


({'_name_or_path': 'output/train_bi-encoder-mnrl-vinai-phobert-base-v2-margin_3.0-2023-08-27_23-13-25/',
  'architectures': ['RobertaModel'],
  'attention_probs_dropout_prob': 0.1,
  'bos_token_id': 0,
  'classifier_dropout': None,
  'eos_token_id': 2,
  'hidden_act': 'gelu',
  'hidden_dropout_prob': 0.1,
  'hidden_size': 768,
  'initializer_range': 0.02,
  'intermediate_size': 3072,
  'layer_norm_eps': 1e-05,
  'max_position_embeddings': 258,
  'model_type': 'roberta',
  'num_attention_heads': 12,
  'num_hidden_layers': 12,
  'pad_token_id': 1,
  'position_embedding_type': 'absolute',
  'tokenizer_class': 'PhobertTokenizer',
  'torch_dtype': 'float32',
  'transformers_version': '4.38.2',
  'type_vocab_size': 1,
  'use_cache': True,
  'vocab_size': 64001,
  '_commit_hash': '84f9d9ada0d1a3c37557398b9ae9fcedcdf40be0'},
 {})

In [ ]:
#@title First test mTGE family model without semantics understanding
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    Trainer,
    TrainingArguments
)
from datasets import load_dataset
import numpy as np
import evaluate

output_dir = "./bkai-embedding-ft"

# 1. Load tokenizer and classification head model
model_name = "bkai-foundation-models/vietnamese-bi-encoder"
num_labels = 7  # binary classification
tokenizer = AutoTokenizer.from_pretrained(model_name,trust_remote_code=True)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=num_labels,trust_remote_code=True)

# 2. Load a classification dataset (IMDB as example)
dataset = load_dataset("tridm/UIT-VSMEC")

# 2. Fixed emotion-to-id mapping
label_list = ['Anger', 'Disgust', 'Enjoyment', 'Fear', 'Other', 'Sadness', 'Surprise']
label2id = {
    'Anger': 0,
    'Disgust': 1,
    'Enjoyment': 2,
    'Fear': 3,
    'Other': 4,
    'Sadness': 5,
    'Surprise': 6
}
id2label = {v: k for k, v in label2id.items()}
label2id = {label: idx for idx, label in enumerate(sorted(label_list))}
print("Label mapping:", label2id)

# 2. Map Emotion strings to integers
def encode_labels(example):
    example["labels"] = label2id[example["Emotion"]]
    return example

dataset = dataset.map(encode_labels)
# 3. Preprocess (tokenize text)
def preprocess_function(examples):
    return tokenizer(examples["Sentence"], truncation=True, padding="max_length", max_length=258)

tokenized_dataset = dataset.map(preprocess_function, batched=True)
tokenized_dataset.set_format("torch", columns=["input_ids", "attention_mask", "labels"])
train_dataset = tokenized_dataset["train"]
test_dataset = tokenized_dataset["test"]
train_subset = train_dataset.shuffle(seed=42).select(range(500))
test_subset = test_dataset.shuffle(seed=42).select(range(200))
# 5. Load metric
accuracy = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)
    return accuracy.compute(predictions=preds, references=labels)

# 6. TrainingArguments
training_args = TrainingArguments(
    output_dir=output_dir,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=1,
    weight_decay=0.01,
    logging_steps=100,
    push_to_hub=False,
    report_to="none",
)

# 7. Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset, # use subset for speed
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

# 8. Train
trainer.train()

# 9. Evaluate
eval_results = trainer.evaluate()
print("Evaluation Results:", eval_results)

# 10. Save the final model
trainer.save_model(output_dir)
tokenizer.save_pretrained(output_dir)


loading file vocab.txt from cache at /home/tminh/.cache/huggingface/hub/models--bkai-foundation-models--vietnamese-bi-encoder/snapshots/84f9d9ada0d1a3c37557398b9ae9fcedcdf40be0/vocab.txt
loading file bpe.codes from cache at /home/tminh/.cache/huggingface/hub/models--bkai-foundation-models--vietnamese-bi-encoder/snapshots/84f9d9ada0d1a3c37557398b9ae9fcedcdf40be0/bpe.codes
loading file added_tokens.json from cache at /home/tminh/.cache/huggingface/hub/models--bkai-foundation-models--vietnamese-bi-encoder/snapshots/84f9d9ada0d1a3c37557398b9ae9fcedcdf40be0/added_tokens.json
loading file special_tokens_map.json from cache at /home/tminh/.cache/huggingface/hub/models--bkai-foundation-models--vietnamese-bi-encoder/snapshots/84f9d9ada0d1a3c37557398b9ae9fcedcdf40be0/special_tokens_map.json
loading file tokenizer_config.json from cache at /home/tminh/.cache/huggingface/hub/models--bkai-foundation-models--vietnamese-bi-encoder/snapshots/84f9d9ada0d1a3c37557398b9ae9fcedcdf40be0/tokenizer_config.js

Label mapping: {'Anger': 0, 'Disgust': 1, 'Enjoyment': 2, 'Fear': 3, 'Other': 4, 'Sadness': 5, 'Surprise': 6}


/home/tminh/anaconda3/lib/python3.12/site-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
PyTorch: setting up devices
/tmp/ipykernel_8577/2437708053.py:76: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
The following columns in the training set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: Emotion, Sentence. If Emotion, Sentence are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running training *****
  Num examples = 5,548
  Num Epochs = 1
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 694
  Nu

Epoch,Training Loss,Validation Loss,Accuracy
1,1.331200,1.268471,0.574315


The following columns in the evaluation set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: Emotion, Sentence. If Emotion, Sentence are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.

***** Running Evaluation *****
  Num examples = 693
  Batch size = 8
Saving model checkpoint to ./gte-seqcls-finetuned/checkpoint-694
Configuration saved in ./gte-seqcls-finetuned/checkpoint-694/config.json
Model weights saved in ./gte-seqcls-finetuned/checkpoint-694/model.safetensors
tokenizer config file saved in ./gte-seqcls-finetuned/checkpoint-694/tokenizer_config.json
Special tokens file saved in ./gte-seqcls-finetuned/checkpoint-694/special_tokens_map.json
added tokens file saved in ./gte-seqcls-finetuned/checkpoint-694/added_tokens.json


Training completed. Do not forget to share your model on huggingface.co/models =)


The following columns in the evaluation set don't have a corresponding a

Saving model checkpoint to ./gte-seqcls-finetuned
Configuration saved in ./gte-seqcls-finetuned/config.json


Evaluation Results: {'eval_loss': 1.2684705257415771, 'eval_accuracy': 0.5743145743145743, 'eval_runtime': 18.016, 'eval_samples_per_second': 38.466, 'eval_steps_per_second': 4.829, 'epoch': 1.0}


Model weights saved in ./gte-seqcls-finetuned/model.safetensors
tokenizer config file saved in ./gte-seqcls-finetuned/tokenizer_config.json
Special tokens file saved in ./gte-seqcls-finetuned/special_tokens_map.json
added tokens file saved in ./gte-seqcls-finetuned/added_tokens.json
tokenizer config file saved in ./gte-seqcls-finetuned/tokenizer_config.json
Special tokens file saved in ./gte-seqcls-finetuned/special_tokens_map.json
added tokens file saved in ./gte-seqcls-finetuned/added_tokens.json


('./gte-seqcls-finetuned/tokenizer_config.json',
 './gte-seqcls-finetuned/special_tokens_map.json',
 './gte-seqcls-finetuned/vocab.txt',
 './gte-seqcls-finetuned/bpe.codes',
 './gte-seqcls-finetuned/added_tokens.json')

In [ ]:
#@title Constractive training with full string target variable
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from transformers import AutoTokenizer, AutoModel
from torch.optim import AdamW
from datasets import load_dataset
from tqdm import tqdm
import random

# ========== 1. Load model and tokenizer ==========
model_name = "bkai-foundation-models/vietnamese-bi-encoder"
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
model = AutoModel.from_pretrained(model_name, trust_remote_code=True)
model.train()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# ========== 2. Load dataset ==========
dataset = load_dataset("tridm/UIT-VSMEC")
train_data = dataset["train"].shuffle(seed=42)  # for speed
eval_data = dataset["test"].shuffle(seed=42)

# ========== 3. Format for contrastive training ==========
label_list = ['Tức giận', 'Kinh tởm', 'Thích thú', 'Sợ hãi', 'Khác', 'Buồn bã', 'Bất ngờ']

class ContrastiveDataset(Dataset):
    def __init__(self, data, tokenizer, label_list, max_length=258):
        self.data = data
        self.tokenizer = tokenizer
        self.labels = label_list
        self.max_length = max_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        sentence = self.data[idx]["Sentence"]
        emotion = self.data[idx]["Emotion"]

        # Positive: (sentence, correct label)
        input_enc = self.tokenizer(sentence, truncation=True, padding="max_length", max_length=self.max_length, return_tensors="pt")
        label_enc = self.tokenizer(emotion, truncation=True, padding="max_length", max_length=self.max_length, return_tensors="pt")

        return {
            "input_ids": input_enc["input_ids"].squeeze(0),
            "attention_mask": input_enc["attention_mask"].squeeze(0),
            "label_ids": label_enc["input_ids"].squeeze(0),
            "label_attention_mask": label_enc["attention_mask"].squeeze(0),
            "emotion": emotion
        }

train_dataset = ContrastiveDataset(train_data, tokenizer, label_list)
eval_dataset = ContrastiveDataset(eval_data, tokenizer, label_list)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
eval_loader = DataLoader(eval_dataset, batch_size=1)

# ========== 4. Contrastive loss function ==========
def cosine_sim(a, b):
    a = F.normalize(a, dim=-1)
    b = F.normalize(b, dim=-1)
    return torch.matmul(a, b.T)

def contrastive_loss(sent_embs, label_embs):
    sim = cosine_sim(sent_embs, label_embs)  # [B, B]
    labels = torch.arange(sim.size(0)).to(sim.device)
    return F.cross_entropy(sim, labels)

# ========== 5. Optimizer ==========
optimizer = AdamW(model.parameters(), lr=2e-5)

# ========== 6. Training Loop ==========
for epoch in range(1):
    model.train()
    total_loss = 0.0
    pbar = tqdm(train_loader, desc=f"Epoch {epoch+1}")

    for batch in pbar:
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        label_ids = batch["label_ids"].to(device)
        label_attention_mask = batch["label_attention_mask"].to(device)

        # Get embeddings ([CLS] token)
        sent_out = model(input_ids=input_ids, attention_mask=attention_mask).last_hidden_state[:, 0]
        label_out = model(input_ids=label_ids, attention_mask=label_attention_mask).last_hidden_state[:, 0]

        loss = contrastive_loss(sent_out, label_out)

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        total_loss += loss.item()
        pbar.set_postfix(loss=loss.item())

    print(f"Epoch {epoch+1} - Avg loss: {total_loss / len(train_loader):.4f}")

# ========== 7. Evaluation ==========
model.eval()
correct = 0

# Precompute label embeddings
with torch.no_grad():
    label_embs = []
    for label in label_list:
        tokens = tokenizer(label, return_tensors="pt", truncation=True, padding=True).to(device)
        label_emb = model(**tokens).last_hidden_state[:, 0]
        label_embs.append(F.normalize(label_emb, dim=-1))
    label_embs = torch.cat(label_embs, dim=0)  # [num_labels, hidden]

for batch in tqdm(eval_loader, desc="Evaluating"):
    input_ids = batch["input_ids"].to(device)
    attention_mask = batch["attention_mask"].to(device)
    true_emotion = batch["emotion"]

    with torch.no_grad():
        sentence_emb = model(input_ids=input_ids, attention_mask=attention_mask).last_hidden_state[:, 0]
        sentence_emb = F.normalize(sentence_emb, dim=-1)
        sims = cosine_sim(sentence_emb, label_embs)  # [1, num_labels]
        pred_idx = torch.argmax(sims, dim=1).item()
        pred_label = label_list[pred_idx]

    if pred_label == true_emotion[0]:
        correct += 1

accuracy = correct / len(eval_dataset)
print(f"\nTop-1 Accuracy: {accuracy:.4f}")


loading file vocab.txt from cache at /home/tminh/.cache/huggingface/hub/models--bkai-foundation-models--vietnamese-bi-encoder/snapshots/84f9d9ada0d1a3c37557398b9ae9fcedcdf40be0/vocab.txt
loading file bpe.codes from cache at /home/tminh/.cache/huggingface/hub/models--bkai-foundation-models--vietnamese-bi-encoder/snapshots/84f9d9ada0d1a3c37557398b9ae9fcedcdf40be0/bpe.codes
loading file added_tokens.json from cache at /home/tminh/.cache/huggingface/hub/models--bkai-foundation-models--vietnamese-bi-encoder/snapshots/84f9d9ada0d1a3c37557398b9ae9fcedcdf40be0/added_tokens.json
loading file special_tokens_map.json from cache at /home/tminh/.cache/huggingface/hub/models--bkai-foundation-models--vietnamese-bi-encoder/snapshots/84f9d9ada0d1a3c37557398b9ae9fcedcdf40be0/special_tokens_map.json
loading file tokenizer_config.json from cache at /home/tminh/.cache/huggingface/hub/models--bkai-foundation-models--vietnamese-bi-encoder/snapshots/84f9d9ada0d1a3c37557398b9ae9fcedcdf40be0/tokenizer_config.js

Epoch 1 - Avg loss: 2.3032


Evaluating: 100%|██████████| 20/20 [00:25<00:00,  1.26s/it]


Top-1 Accuracy: 0.0000
